In [1]:
import sys,os
dev_mode = True
preset = 'python'

if dev_mode:
    try:
        install_prefix = os.environ['CMAKE_INSTALL_PREFIX']
        print(f'Using CMAKE_INSTALL_PREFIX={install_prefix}')
    except KeyError:
        install_prefix = f'../../build/{preset}/install'
print(f'Using install_prefix={install_prefix}')
if dev_mode and os.path.exists(f'{install_prefix}/lib/python3/dist-packages'):
    sys.path.insert(0, os.path.abspath(f'{install_prefix}/lib/python3/dist-packages'))
print(sys.path)

Using install_prefix=../../build/python/install
['/scratch/prudhomm/Devel/feelpp.quickfix/build/python/install/lib/python3/dist-packages', '/scratch/prudhomm/Devel/feelpp.quickfix/python/notebooks', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/u2/prudhomm/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/local/lib/python3/dist-packages/', '/usr/lib/python3/dist-packages', '/usr/lib/petsc/lib/python3/dist-packages', '/usr/lib/slepc/lib/python3/dist-packages', '/usr/lib/python3/dist-packages/IPython/extensions', '/home/u2/prudhomm/.ipython']


In [2]:
import feelpp
from feelpp.meshmover import *
from feelpp.integrate import integrate
from feelpp.measure import measure
import feelpp.toolboxes.core as core
from feelpp.toolboxes.cfpdes import *
import numpy as np
print(f"feelpp loaded from {feelpp.__file__}")

feelpp loaded from /scratch/prudhomm/Devel/feelpp.quickfix/build/python/install/lib/python3/dist-packages/feelpp/__init__.py


In [3]:
sys.argv = ['notebook-mesh']
e = feelpp.Environment(sys.argv,config=feelpp.localRepository(sys.argv[0]),opts = core.toolboxes_options("coefficient-form-pdes", "cfpdes"))

In [4]:
geo = {
    '2': feelpp.create_rectangle(),
    '3': feelpp.create_box(),
}
 

In [5]:
mesh_name, dim, e_meas, e_s_1, e_s_2, e_s_bdy = geo['2']
mesh = feelpp.load(feelpp.mesh(dim=dim), mesh_name, 0.1)

Xh = feelpp.functionSpace(mesh=mesh,space="Pchv")
u = Xh.element()
M=measure(range=feelpp.elements(mesh))
assert(abs(M-2)<1e-10)

In [10]:
laplacian={
    "Name": "Laplacian",
    "ShortName": "Laplacian",
    "Models":
    {
        "cfpdes":{ "equations":"laplace" },
        "laplace":{
            "setup":{
                "unknown":{
                    "basis":"Pchv1",
                    "name":"u",
                    "symbol":"u"
                },
                "coefficients":{ "c":"1", "f":"{1,1}"}
            }
        }
    },
    "Materials": { "Omega": { "markers":"Omega" } },
    "BoundaryConditions":
    {
        "laplace":
        {
            "Dirichlet":
            {
                "GammaMove":
                {
                    "markers":["Gamma_1"],
                    "expr":"{0,amplitude*x*(1-x)+0.1}:x:amplitude"
                },
                "zero":
                {   
                    "markers":["Gamma_2","Gamma_3","Gamma_4"],
                    "expr":"{0,0}"
                }
            }
        }
    },
    "PostProcess":
    {
        "cfpdes":
        {
            "Exports":
            {
                "fields":["all"]
            }
        }
    }
}


In [ ]:
toolbox=cfpdes(dim=2)
toolbox.setMesh(mesh)
toolbox.setModelProperties(laplacian)
toolbox.addParameterInModelProperties("amplitude",2.)

#
toolbox.init(buildModelAlgebraicFactory=True)
toolbox.solve()
toolbox.exportResults()

In [ ]:
assert(abs(M-2)<1e-10)
newmesh = meshMove(mesh,u)
M=measure(range=feelpp.elements(newmesh))   

# Equal to 8 in 2d and 3d
assert(abs(M-8)<1e-10)